<div align="center"> <h1> 
Bamboost<br/>
🐼🐼🐼🐼
</h1> </div>

Bamboost is a Python library built for datamanagement using the HDF5 file format.
bamboost stands for a lightweight shelf which will boost your efficiency and which
will totally break if you load it heavily. Just kidding, bamboo can fully carry pandas. 

### Data architecture
<img src="./assets/data_architecture.excalidraw.png" >

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from bamboost import Manager

from lattice_model import LatticeModel

## Parametric study
Let's run many simulations on the lattice model from before.

Remember that we have 3 parameters:
- `N` controls the amount of nodes
- `E` is the young's modulus which we will keep constant and equal to $1$ for brevity
- `randomness` is a parameter we use to randomly move the points away from their position in a regular grid. The larger it is, the further the points are moved from the initial position.

The young's modulus is very uninteresting in this case, so we'll limit ourselfs on the number of nodes and the perturbation of nodal positions.



### Model execution
We imitate some program which computes the model with a simple function.
It will take a bamboost `simulation` object as input and run all the computations.

Let's first build this function. It does the same as we have done in the previous tutorial.
Contrary to before, we will also store the loadsteps as an input parameter of the simulation.
To do so, we add the numpy array with the loadsteps to the parameter dictionary.

In [ ]:
def run_simulation(sim):
    """Execute the model computation.
    
    Args:
        sim: the simulation object
    """

    # RETRIEVE THE PARAMETERS
    parameters = sim.parameters

    # setup the lattice model
    model = LatticeModel(parameters['E'], parameters['N'], parameters['randomness'])

    with sim:
        # WRITE THE MESH:
        sim.add_mesh(model.coordinates, model.connectivity)
        
        # WRITE METADATA:
        sim.add_metadata()
        
        # MAKE A NOTE FOR LATER YOU:
        sim.change_note('we are on the mountain')

        # model: loop through the loadsteps
        loadsteps = parameters['loadsteps']

        for disp, change in zip(loadsteps, np.diff(loadsteps, prepend=0)):
            model.move_top(change)
            model.solve()
            
            # WRITE THE NODAL DISPLACEMENTS FOR THIS STEP (use the displacement value as "time"):
            sim.add_field('nodal_displacement', model.displacements, time=disp)
            
            # WRITE THE NODAL FORCES FOR THIS STEP:
            sim.add_field('nodal_force', model.forces, time=disp)

            # WRITE THE TOP DISPLACEMENT, AND THE BOTTOM REACTION FORCE:
            sim.add_global_field('top_disp', disp)
            sim.add_global_field('bottom_force_x', model.get_force_bottom()[0])
            sim.add_global_field('bottom_force_y', model.get_force_bottom()[1])
            
            # bamboost: end the step, will increase an internal step counter
            sim.finish_step()
        
    return 'Success yey'


### Creation of simulations
We want to run simulations for the following parameters:
- `N`: 10, 20, 30
- `randomness`: 0.1, 0.2, 0.3

First, we create a bunch of simulations.

In [ ]:
# CREATE A NEW DATABASE
db = Manager('database')
db.df

In [ ]:
from itertools import product

# DEFINE THE POSSIBLE PARAMETERS
vec_N = [10, 20, 30]
vec_random = [.1, .2, .3]

# We can use product from itertools to get all combinations from the two arrays
for N, randomness in product(vec_N, vec_random):
    
    # CREATE PARAMETER DICTIONARY
    params = {
        'E': 1,
        'N': N,
        'randomness': randomness,
        'loadsteps': np.linspace(0, 2, 20),
    }

    # CREATE SIMULATION IN DATABASE
    db.create_simulation(parameters=params)

In [ ]:
# see that we now have 9 simulations in our database
db.df

### Execution of simulations
Next, we can execute all simulations.

In [ ]:
# LOOP THROUGH ALL SIMULATIONS:
for sim in db.sims(db.df.status=='Initiated', return_writer=True):
    
    print(f'Starting simulation {sim.uid}', flush=True)
    
    # CALL RUN FUNCTION:
    run_simulation(sim)

In [ ]:
# see that now all are finished (it has the note)
db.df

### Data mining
Now that we have all our runs completed, we can look at results.

In [ ]:
# PLOT THE PARAMETRIC SPACE:
db = Manager('database')
db.df.plot('N', 'randomness', kind='scatter')

In [ ]:
from lattice_model import plot_lattice

# PLOT THE DEFORMED LATTICE FOR A SPECIFIC SIM:
N = 10
randomness = 0.1
sim = db.sims((db.df.N==N) & (db.df.randomness==randomness))[0]
print(sim.parameters)

plot_lattice(*sim.mesh, sim.data['nodal_displacement'][10])

In [ ]:
fig, axes = plt.subplots(1, 2)
ax, bx = axes
ax.set(xlabel='N', ylabel='Fx')
bx.set(xlabel='randomness', ylabel='Fx')

# PLOT THE FINAL HORIZONTAL FORCE VS N:
ni = [sim.parameters['N'] for sim in db]
fxi = [sim.globals['bottom_force_x'].iloc[-1] for sim in db]
ax.scatter(ni, fxi, alpha=.5, s=100)

# PLOT THE FINAL HORIZONTAL FORCE VS RANDOMNESS:
for sim in db:
    cb = bx.scatter(
        sim.parameters['randomness'],
        sim.globals['bottom_force_x'].iloc[-1],
        alpha=.5, s=100,
        color='k',
    )
fig.tight_layout()

In [ ]:
# PLOT THE AVERAGE ABSOLUTE NODAL FORCE FOR ALL RUNS OF N=20:

sims = db.sims(db.df.N==20, sort='randomness')

for sim in sims:
    plt.plot(sim.globals['top_disp'],
             np.abs(sim.data['nodal_force'][:, :, 0]).mean(axis=1),
             label=f"randomness = {sim.parameters['randomness']}",
    )
plt.legend()


In [ ]:
# GET ALL SIMS IN A LIST
sims = db.sims()

# SORT THE SIMS BY THE ABSOLUTE NODAL DISPLACEMENT OF THE NODE CLOSEST TO (x,y)=(.5,.5):
def get_center_displacement(sim):
    center_node_idx = np.argmin(np.linalg.norm(sim.mesh[0] - np.array([.5, .5]), axis=1))    
    center_displacement = sim.data['nodal_displacement'][-1, center_node_idx]
    center_displacement_norm = np.linalg.norm(center_displacement)
    return center_displacement_norm

sorted_sims = sorted(sims, key=lambda sim: key_function(sim))

# PLOT THE CENTER NODE DISPLACEMENT
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(
    [get_center_displacement(sim) for sim in sorted_sims]
)

# CHECK WHETHER THIS VALUE CORRELATES WITH THE RANDOMNESS PARAMETER
bx = ax.twinx()
bx.plot(
    [sim.parameters['randomness'] for sim in sorted_sims],
    c='r',
)